In [1]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [2]:
import models
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [3]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = models.get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder'

# model is actually in "serialized_models/{model_fname}"
# model_fname = 'rel_tsel_ref_dial_model/default_0_best.th'

# # valid_lang_loss 2.2762        valid_select_loss 0.8293        valid_select_acc 0.6859 valid_ref_loss 0.2897
# model_fname = 'rel2_tsel_ref_dial_model/hierarchical_rel-enc-2_rel-pool-max_mask-pad_1_best.th'

# # valid_lang_loss 2.2609        valid_select_loss 0.9497        valid_select_acc 0.6291         valid_ref_loss 0.3191
# model_fname = 'rel2_tsel_ref_dial_model/hierarchical_rel-enc-2_rel-pool-max_mask-pad_feed-ctx-attend_1_best.th'

# valid_lang_loss 
# # valid_lang_loss 2.2544        valid_select_loss 0.8247        valid_select_acc 0.7062         valid_ref_loss 0.2852
# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_feed-context-attend-separate_1_best.th'

# # valid_lang_loss 2.2448        valid_select_loss 0.9233        valid_select_acc 0.6455         valid_ref_loss 0.3180
# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_selection-attention_feed-context-attend-separate_no-word-attention_1_best.th'

# model_fname = 'rel2_tsel_ref_dial_model_no-share/hierarchical_mask-pad_selection-attention_feed-context-attend-separate_1_best.th'

# valid_lang_loss 2.2435        valid_select_loss 1.0267        valid_select_acc 0.6358         valid_ref_loss 0.3528
# model_fname = 'rel2_tsel_ref_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_no-word-attention_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'

model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_feed-context-attend_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

In [4]:
# supervise attn
model_fname = 'rel2_dial_model_separate/hierarchical_only-self_supervise-feed-attn_mask-pad_feed-context-attend_1_best.th'

In [5]:
## mark mentioned
model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mark-dots-mentioned_mask-pad_1_best.th'
## this one's running
model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mark-dots-mentioned_mask-pad_rel-2_feed-context-attend_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mark-dots-mentioned_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_1_best.th'

In [6]:
# CONSTRAIN ATTENTION
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_1_best.th'

# has diverse outputs, but doesn't attend to meaningful things
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_ablate-properties_1_best.th'

# feed attention is peaked, word attention doesn't attend to meaningful things
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_feed-context-attend_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_feed-context-attend_no-word-attention_ablate-properties_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_1_best.th'
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_mask-pad_max-pool_rel-2_feed-context-attend_1_best.th'

# valid_lang_loss 1.0645
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_sigmoid_mask-pad_feed-context-attend_1_best.th'

# valid_lang_loss 1.0674
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_sigmoid_mask-pad_1_best.th'

# valid_lang_loss 1.0678
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_sigmoid_mask-pad_rel-2_feed-context-attend_1_best.th'

# valid_lang_loss 1.0654
# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_sigmoid_mask-pad_rel-2_1_best.th'

# valid_lang_loss 1.0635
model_fname = 'rel2_dial_model_separate/hierarchical_only-self_constrain-attn_sigmoid_ria_pic_mask-pad_rel-2_1_best.th'

In [8]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

dataset data/onecommon/train_reference_1.txt, total 545056, unks 4302, ratio 0.79%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 686, ratio 0.99%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 636, ratio 0.95%


In [9]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [10]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [11]:
len(scenarios1)

9000

In [12]:
len(scenarios2)

4500

In [13]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [14]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [15]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [16]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [17]:
corpus

In [18]:
model = utils.load_model(model_fname, map_location='cpu')

In [19]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

In [20]:
validset_stats

{'n': 74030, 'nonpadn': 69348}

In [21]:
import torch

In [22]:
from agent import RnnAgent

/home/dfried/anaconda2/envs/onecommon/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [23]:
from selfplay import make_parser

In [24]:
parser = make_parser()
# args = parser.parse_args('--temperature 0.25'.split())
args = parser.parse_args('--temperature 0.01'.split())

In [25]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [26]:
def is_selection(out):
    return '<selection>' in out

In [27]:
# display

In [28]:
from dialog import DialogLogger

In [53]:
from IPython.display import SVG, display, HTML

In [30]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [31]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn)])

In [32]:
pdb on

Automatic pdb calling has been turned ON


In [33]:
# training emulation

In [34]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

In [35]:
trainset[0].scenario_ids[0] in train_scenarios

True

In [36]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [37]:
for ix in range(len(trainset)):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
#     if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


In [38]:
# training emulation
scenario = train_scenarios[scenario_id]
agent = alice
agent.feed_context(ctx.flatten())
agent.real_ids = real_ids
agent.agent_id = agents[0]
    
samples = 10
    
writer = alice
conv = []
speaker = []
words_left = 5000

sentence_ix = 0

assert len(inpts) == len(tgts)
    
while sentence_ix < len(inpts):
    words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]
#     for _ in range(samples):
#         pred_outs, _, _, _, extra = writer.model.write(writer.ctx_h, writer.lang_h, words_left, writer.args.temperature)
#         print('{}\t{}'.format(writer.agent_id, ' '.join(writer._decode(outs.narrow(0,1,outs.size(0)-1), writer.model.word_dict))))
    print(' '.join(words_og))
    if ref_inpts[sentence_ix] is not None:
        dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
        display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
    else:
        dots_mentioned = None
    out = writer.write(
        max_words=words_left,
        force_words=[words_og],
        start_token=words_og[0],
        dots_mentioned=dots_mentioned
    )
#     out = writer.write(max_words=words_left)
    
    if writer.extras[-1]['feed_ctx_attn_prob'] is not None:
        display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
    if 'word_ctx_attn_prob_mean' in writer.extras[-1]:
        display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
    for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
        display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
#     reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
#     writer, reader = reader, writer
    sentence_ix += 1

YOU: i see a dark grey dot with a slightly smaller darker dot below it <eos>
dots_mentioned: [1. 1. 0. 0. 0. 0. 0.]


word attention mean: [0.53589165 0.52977246 0.         0.         0.         0.
 0.        ]


t 0: i: [0.8211394 0.7937835 0.        0.        0.        0.        0.       ]


t 1: see: [0.45094448 0.4484574  0.         0.         0.         0.
 0.        ]


t 2: a: [0.978592  0.9823231 0.        0.        0.        0.        0.       ]


...

t 13: it: [0.3041241 0.3016892 0.        0.        0.        0.        0.       ]


t 14: <eos>: [0.1890018  0.18899901 0.         0.         0.         0.
 0.        ]


1	i see a dark grey dot with a slightly smaller darker dot below it <eos>

THEM: i think i have that too . above the larger one and to its left i have a much smaller medium gray dot <eos>


AssertionError: 

> /home/dfried/projects/onecommon/aaai2020/experiments/models/rnn_reference_model.py(329)_language_conditioned_dot_attention()
    327         assert lang_hs.dim() == 3
    328 
--> 329         assert dots_mentioned is not None
    330 
    331         seq_len, bsz, _ = lang_hs.size()

ipdb> q


In [39]:
import json
from utils import ContextGenerator

In [40]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [41]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [48]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 2:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_lc9VK9nxUuXpyiRP


In [56]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
dot_mention_samples = 2
utt_samples = 5
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000

num_dots_to_mention = 0

temperature_override = 0.5

def make_dots_mentioned():
    # fixed
#     return torch.BoolTensor([[1,0,0,1,0,0,0]])

    # random 
    mentioned = torch.zeros(1,7).bool()
    for _ in range(num_dots_to_mention):
        ix = torch.randint(7, (1,))
        mentioned[:,ix] = 1
    return mentioned

display_context = False

while True:
    for dm in range(dot_mention_samples):
        dots_mentioned = make_dots_mentioned()
        if display_context:
            display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
        for us in range(utt_samples):
            outs, _, _, _, extra = writer.model.write(writer.ctx_h, writer.lang_h, words_left, temperature_override, dots_mentioned=dots_mentioned)
            print('{}\t{}'.format(writer.agent_id, ' '.join(writer._decode(outs.narrow(0,1,outs.size(0)-1), writer.model.word_dict))))
        print()
        
    dots_mentioned = make_dots_mentioned()
    if display_context:
        display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
    out = writer.write(max_words=words_left, dots_mentioned=dots_mentioned, temperature_override=temperature_override)
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
    if display_context and writer.extras[-1].get('feed_ctx_attn_prob', None) is not None:
        display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
    if display_context and writer.extras[-1].get('word_ctx_attn_prob_mean', None) is not None:
        display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['dot_attention2s']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print('-'*60)
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out, dots_mentioned=dots_mentioned)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

0	hello ! wheres your dot ? <eos>
0	hello which dot would you like to try for ? <eos>
0	hi ! <eos>
0	hello ! wheres your dot ? <eos>
0	hello ! wheres your dot ? <eos>

0	hello which dot would you like to try for ? <eos>
0	hello which dot would you like to try for ? <eos>
0	hello ! wheres your dot ? <eos>
0	hello ! wheres your dot ? <eos>
0	hello ! wheres your dot ? <eos>

0	hello ! where 's your dot ? <eos>

0	hello ! where 's your dot ? <eos>
------------------------------------------------------------
1	do you have any dots in a diagonal line ? <eos>
1	i have one medium sized very dark dot <eos>
1	hi , i have 1 lonely dot . <eos>
1	i have one large dot <eos>
1	i have one medium sized dot at the top <eos>

1	hi <eos>
1	i have one lonely dot <eos>
1	i have one lonely dot <eos>
1	i have 1 lonely dot at the bottom . <eos>
1	i have 1 lonely dot that is almost black <eos>

1	do you see a lonely gray dot <eos>

1	do you see a lonely gray dot <eos>
-------------------------------------------